In [ ]:
# Automating Product Data Entry via Telegram and Google Sheets

In [ ]:
pip install gspread oauth2client python-telegram-bot nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.7/662.7 kB 10.6 MB/s eta 0:00:00


In [ ]:
import asyncio
import nest_asyncio
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
nest_asyncio.apply()

In [ ]:
# Google Sheets Authentication
def connect_to_google_sheet(sheet_name):
    # Defining the scope of the API
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name("/content/drive/MyDrive/Colab Notebooks/task1.json", scope)
    client = gspread.authorize(credentials)
    sheet = client.open(sheet_name).sheet1
    return sheet

In [ ]:
# Telegram Bot Handlers
async def start(update: Update, context: CallbackContext):
    await update.message.reply_text("Welcome! Send me product details in the format: Product Name, Price")


In [ ]:
async def add_data(update: Update, context: CallbackContext):
    # Extract message content
    message = update.message.text
    try:
        # Split the message to extract product and price
        product, price = map(str.strip, message.split(","))
        price = float(price)

        # Connect to Google Sheet
        sheet = connect_to_google_sheet("Task1")

        # Add the product and price to the sheet
        sheet.append_row([product, price])
        await update.message.reply_text(f"Added: {product} - Rs {price:.2f}")
    except ValueError:
        await update.message.reply_text("Invalid format! Please send data in the format: Product Name, Price")
    except Exception as e:
        await update.message.reply_text(f"An error occurred: {e}")

In [ ]:
async def main():
    # Telegram bot token
    BOT_TOKEN = "7805995554:AAHwmBRGMXnO3ConPCh72J6wZX6UarS3IxA"

    # Initialize the bot application
    application = Application.builder().token(BOT_TOKEN).build()

    # Add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, add_data))

    # Start the bot
    print("Bot is running...")
    await application.run_polling()

In [ ]:
# Run the bot
await main()

Bot is running...
